In [22]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display

n = 100
pixel_val_max = 255
pixel_val_min = 0


sums=[[]]
# Define the neural network model
model = Sequential([
    Dense(3, activation='sigmoid', input_shape=(2,)),
    Dense(3, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['accuracy'])

# Generate training data
X = np.random.uniform(pixel_val_min, pixel_val_max, (1000, 2))
sums=[[]]
def S1(X, n):
    y = np.zeros((X.shape[0], 3))
    for i, (x1, x2) in enumerate(X):
        if x1 > x2 and x1 < (pixel_val_min+n):
            y[i, 0] = 1
        elif x1 < x2 and x2 > (pixel_val_max - n):
            y[i, 1] = 1
        else:
            y[i, 2] = 1
    print(y)
    sums.append(y)
    return y

def S2(X, n):
    y = np.zeros((X.shape[0], 3))
    for i, (x1, x2) in enumerate(X):
        if x1 < x2:
            y[i, 0] = 1
        elif x1 < x2:
            y[i, 1] = 1
        else:
            y[i, 2] = 1
    print(y)
    return y

import os
import json
import numpy as np


def s4(folder_path):
    
    def encode_colors(color):
        global black_count
        global grey_count
        global white_count
        if color == "black":
            black_count += 1
            return [1, 0, 0]
        elif color == "grey":
            grey_count+=1
            return [0, 1, 0]
        else:
            white_count+=1
            return [0, 0, 1]

    # Read JSON files
    all_data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):
            with open(os.path.join(folder_path, file_name), "r") as f:
                file_data = json.load(f)
                all_data.extend(file_data)

    # Preprocess data
    X = []
    y = []
    for data in all_data:
        X.append([data["X1"], data["X2"]])
        y.append(encode_colors(data["selectedColor"]))

    X = np.array(X)
    y = np.array(y)

    return X, y

# humanx,humany = s4("/Users/jivitesh/Library/Containers/com.example.dataCollecter1/Data/Documents/")

# print (humans);
y = S1(X, n)

# # Train the model
model.fit(X, y, epochs=1000, verbose=1)


def create_plot(weight_updates, bias_updates):
    for i, layer in enumerate(model.layers):
        weights = layer.get_weights()
        weights[0] += weight_updates[i]
        weights[1] += bias_updates[i]
        layer.set_weights(weights)

    x_vals = np.linspace(pixel_val_min, pixel_val_max, 50)
    y_vals = np.linspace(pixel_val_min, pixel_val_max, 50)
    X_grid, Y_grid = np.meshgrid(x_vals, y_vals)
    input_grid = np.column_stack((X_grid.ravel(), Y_grid.ravel()))
    output_grid = model.predict(input_grid)
    print(output_grid);
    fig = make_subplots(rows=1, cols=4, specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}, {'type': 'scatter3d'}]])

    trace1 = go.Surface(x=X_grid, y=Y_grid, z=output_grid[:, 0].reshape(X_grid.shape), showscale=False,
                        hovertemplate="Input Neuron 1 : %{x} <br>Input Neuron 2 : %{y} <br>Output Neuron 1: %{z} <br> Output Neuron 2: %{customdata[0]} <br>Output Neuron 3: %{customdata[1]}<extra></extra>",
                        customdata=np.column_stack((output_grid[:, 1], output_grid[:, 2])).reshape(X_grid.shape[0], X_grid.shape[1], 2))
                        # customdata=np.column_stack((output_grid[:, 1], output_grid[:, 2], np.ones_like(output_grid[:, 0]) * 42, np.ones_like(output_grid[:, 0]) * 99)).reshape(X_grid.shape[0], X_grid.shape[1], 4))

        
    trace2 = go.Surface(x=X_grid, y=Y_grid, z=output_grid[:, 1].reshape(X_grid.shape), showscale=False,
                        hovertemplate="Input Neuron 1: %{x}, Input Neuron 2: %{y}<br>Output Neuron 1: %{customdata[0]}<br>Output Neuron 2: %{z}<br>Output Neuron 3: %{customdata[1]}<extra></extra>",
                        customdata=np.column_stack((output_grid[:, 0], output_grid[:, 2])).reshape(X_grid.shape[0], X_grid.shape[1], 2))
    trace3 = go.Surface(x=X_grid, y=Y_grid, z=output_grid[:, 2].reshape(X_grid.shape), showscale=False,
                        hovertemplate="Input Neuron 1: %{x}, Input Neuron 2: %{y}<br>Output Neuron 1: %{customdata[0]}<br>Output Neuron 2: %{customdata[1]}<br>Output Neuron 3: %{z}<extra></extra>",
                        customdata=np.column_stack((output_grid[:, 0], output_grid[:, 1])).reshape(X_grid.shape[0], X_grid.shape[1], 2))
    
    trace4 = go.Scatter3d(x=input_grid[:, 0], y=input_grid[:, 1], z=output_grid[:, 0], mode='markers', marker=dict(size=3, color='red'), name='Output Neuron 1')
    trace5 = go.Scatter3d(x=input_grid[:, 0], y=input_grid[:, 1], z=output_grid[:, 1], mode='markers', marker=dict(size=3, color='green'), name='Output Neuron 2')
    trace6 = go.Scatter3d(x=input_grid[:, 0], y=input_grid[:, 1], z=output_grid[:, 2], mode='markers', marker=dict(size=3, color='blue'), name='Output Neuron 3')


    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=1, col=2)
    fig.add_trace(trace3, row=1, col=3)
    
    fig.add_trace(trace4, row=1, col=4)
    fig.add_trace(trace5, row=1, col=4)
    fig.add_trace(trace6, row=1, col=4)
    
    min_z = min(output_grid.min(), output_grid.min(), output_grid.min())
    max_z = min(max(output_grid.max(), output_grid.max(), output_grid.max()),1) 
    
    fig.update_layout(hovermode="x unified", hoverdistance=50,
                      scene=dict(xaxis_title="Input Neuron 1", yaxis_title="Input Neuron 2", zaxis_title="Output Neuron 1", zaxis_range=[min_z, max_z]),
                      scene2=dict(xaxis_title="Input Neuron 1", yaxis_title="Input Neuron 2", zaxis_title="Output Neuron 2",  zaxis_range=[min_z, max_z]),
                      scene3=dict(xaxis_title="Input Neuron 1", yaxis_title="Input Neuron 2", zaxis_title="Output Neuron 3", zaxis_range=[min_z, max_z]),
                      scene4=dict(xaxis_title="Input Neuron 1", yaxis_title="Input Neuron 2", zaxis_title="Output Neurons"))

    return fig

def plot(weight_update, bias_update):
    weight_updates = [np.ones((2, 3)) * weight_update, np.ones((3, 3)) * weight_update]
    bias_updates = [np.ones(3) * bias_update, np.ones(3) * bias_update]
    fig = create_plot(weight_updates, bias_updates)
    fig.show()


weight_slider = widgets.FloatSlider(min=-1, max=1, step=0.1, value=0, description="Weight Update")
bias_slider = widgets.FloatSlider(min=-1, max=1, step=0.1, value=0, description="Bias Update")

print("These are the ",sums)
print("This is black, grey and white",black_count, grey_count, white_count)
widgets.interact(plot, weight_update=weight_slider, bias_update=bias_slider)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
Epoch 1/1000
32/32 [==============================] - 0s 603us/step - loss: 0.2731 - accuracy: 0.3740
Epoch 2/1000
32/32 [==============================] - 0s 594us/step - loss: 0.2021 - accuracy: 0.6020
Epoch 3/1000
32/32 [==============================] - 0s 553us/step - loss: 0.1875 - accuracy: 0.6020
Epoch 4/1000
32/32 [==============================] - 0s 477us/step - loss: 0.1823 - accuracy: 0.6020
Epoch 5/1000
32/32 [==============================] - 0s 523us/step - loss: 0.1800 - accuracy: 0.6020
Epoch 6/1000
32/32 [==============================] - 0s 493us/step - loss: 0.1787 - accuracy: 0.6020
Epoch 7/1000
32/32 [==============================] - 0s 502us/step - loss: 0.1781 - accuracy: 0.6020
Epoch 8/1000
32/32 [==============================] - 0s 515us/step - loss: 0.1777 - accuracy: 0.6020
Epoch 9/1000
32/32 [==============================] - 0s 527us/step - loss: 0.1761 - accuracy: 0.6020
Epoc

interactive(children=(FloatSlider(value=0.0, description='Weight Update', max=1.0, min=-1.0), FloatSlider(valu…

<function __main__.plot(weight_update, bias_update)>